In [1]:
# Imorting libraries
import sys
import logging
import datetime
import time
import os

In [2]:
# Install Azure eventhub python library 
!pip install azure-eventhub

    100% |████████████████████████████████| 61kB 3.0MB/s ta 0:00:01
    100% |████████████████████████████████| 2.8MB 341kB/s eta 0:00:01


In [3]:
# Import Azure eventhub library for python
from azure.eventhub import *

In [30]:
connstr = "Endpoint=sb://evtsentiment.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=702sb4SwBuE2ZoqHZUQrQxL/HLjKFN/04JlPplKI8D8="
client = EventHubClient.from_connection_string(connstr, eventhub='evethubsentiment')
try:
    #if not ADDRESS:
    #    raise ValueError("No EventHubs URL supplied.")

    # Create Event Hubs client
    # client = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)
    sender = client.add_sender(partition="0")
    client.run()
    try:
        start_time = time.time()
        for i in range(3):
            print("Sending message: test message {}".format(i))
            sender.send(EventData(str(i)))
    except:
        raise
    finally:
        end_time = time.time()
        client.stop()
        run_time = end_time - start_time
        print("Runtime: {} seconds".format(run_time))

except KeyboardInterrupt:
    pass

Sending message: test message 0
Sending message: test message 1
Sending message: test message 2
Runtime: 0.18042349815368652 seconds


In [27]:
eh_hub_info = client.get_eventhub_info()
partitions = eh_hub_info["partition_ids"]
print (eh_hub_info)

{'name': 'evethubsentiment', 'type': 'com.microsoft:eventhub', 'created_at': datetime.datetime(2018, 12, 16, 20, 46, 27, 413000), 'partition_count': 2, 'partition_ids': ['0', '1']}


In [16]:
# tidying things up a bit, we can rely on this to send data to event hub
connstr = "Endpoint=sb://evtsentiment.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=702sb4SwBuE2ZoqHZUQrQxL/HLjKFN/04JlPplKI8D8="
client = EventHubClient.from_connection_string(connstr, eventhub='evethubsentiment')
try:
    #if not ADDRESS:
    #    raise ValueError("No EventHubs URL supplied.")
    # Create Event Hubs client
    # client = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)
    sender = client.add_sender(partition="0")
    client.run()
    try:
        start_time = time.time()
        def send_data(ehMessage):
            # Add your logic here to parse the data coming from twitter
            sender.send(EventData(ehMessage))
            print("Sending message: %s" % ehMessage)
        # put the function that reads twitter data here
        for i in range(3):
            send_data("sending test message : %s" % str(i))
    except:
        raise
    finally:
        end_time = time.time()
        client.stop()
        run_time = end_time - start_time
        print("Runtime: {} seconds".format(run_time))
except KeyboardInterrupt:
    pass

Sending message: sending test message : 0
Sending message: sending test message : 1
Sending message: sending test message : 2
Runtime: 0.29123830795288086 seconds


In [18]:
# Receive data from EventHub
CONSUMER_GROUP = "$default"
OFFSET = Offset("-1")
PARTITION = "0"

total = 0
last_sn = -1
last_offset = "-1"
#client = EventHubClient(ADDRESS, debug=False, username=USER, password=KEY)
client = EventHubClient.from_connection_string(connstr, eventhub='evethubsentiment')
try:
    receiver = client.add_receiver(CONSUMER_GROUP, PARTITION, prefetch=5000, offset=OFFSET)
    client.run()
    start_time = time.time()
    for event_data in receiver.receive(timeout=100):
        last_offset = event_data.offset.value
        last_sn = event_data.sequence_number
        msg_body = event_data.body_as_str()
        # print("Received message : {}, {}".format(last_offset, last_sn))
        total += 1
        print("Partition {}, Body {}, Received {}, sn={} offset={}".format(
            PARTITION,
            msg_body,
            total,
            last_sn,
            last_offset))

    end_time = time.time()
    client.stop()
    run_time = end_time - start_time
    print("Received {} messages in {} seconds".format(total, run_time))

except KeyboardInterrupt:
    pass
finally:
    client.stop()

Partition 0, Body Message 0, Received 1, sn=0 offset=0
Partition 0, Body Message 2, Received 2, sn=1 offset=48
Partition 0, Body 0, Received 3, sn=2 offset=96
Partition 0, Body 1, Received 4, sn=3 offset=144
Partition 0, Body 2, Received 5, sn=4 offset=192
Partition 0, Body 0, Received 6, sn=5 offset=240
Partition 0, Body 1, Received 7, sn=6 offset=288
Partition 0, Body 2, Received 8, sn=7 offset=336
Partition 0, Body 0, Received 9, sn=8 offset=384
Partition 0, Body 1, Received 10, sn=9 offset=432
Partition 0, Body 2, Received 11, sn=10 offset=480
Partition 0, Body 0, Received 12, sn=11 offset=528
Partition 0, Body 1, Received 13, sn=12 offset=576
Partition 0, Body 2, Received 14, sn=13 offset=624
Partition 0, Body 0, Received 15, sn=14 offset=672
Partition 0, Body 1, Received 16, sn=15 offset=720
Partition 0, Body 2, Received 17, sn=16 offset=768
Partition 0, Body 0, Received 18, sn=17 offset=816
Partition 0, Body 1, Received 19, sn=18 offset=864
Partition 0, Body 2, Received 20, sn=1